In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



In [0]:
%%time
# read the data
df = pd.read_excel('data.xlsx')

# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'primarycolor'].drop(columns = ['attribute_name'])

# combining similar category attribute values and removing spaces
# df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semi fit' else x)
# df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fitted tailor' else x)
# df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semifitte' else x)
# df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fittedtailore' else x)
# df['attribute_value'] = df['attribute_value'].apply(lambda x: 'straightregular' if x == 'straight regular' else x)

CPU times: user 11.8 s, sys: 288 ms, total: 12.1 s
Wall time: 14.7 s


In [0]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value
2852,01E603S8HDAQM7HKKYDYQQR9R9,simon miller,tone high waist wide leg jean,"light wash cuff raw , fraying edge create cool...",unknown,blue
2857,01E5ZWWZG54Q290WNC1D2D92EV,l c,cicely pintuck linen blend maxi dress,pintucked detail create flattering empire wais...,unknown,pinks
2859,01DVA4YHJQNP81XPB75JBGQ749,jimmy choo,erin suede slingback point toe flat,slight heel navy suede goat buckle fastening s...,shoe flat shoe point toe flat,navy
2861,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunt season,leather panel raffia shoulder bag,"handwoven local artisan , hunt season 's shoul...",woman bags shoulder bag,beige
2866,01DTJ8DQ5SQYBS5HX6TDH2BSJP,hunt season,leather panel raffia shoulder bag,"handwoven local artisan , hunt season 's shoul...",woman bags shoulder bag,darkbrown


In [0]:
# creating a feature combining brand, productname, description and brand category

df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value']).values

### LSTM

In [0]:
# creating a function to create uni-grams and tokenizing data
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
# creating a function to create bi-grams and tokenizing data
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
Tokenizer()

In [0]:
# creating tf-idf vectors from uni-gram encoding
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
# creating tf-idf vectors from bi-gram encoding
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [0]:
X1.shape

(5306, 155)

In [0]:
# concatenating uni-gram and bi-gram
X = np.concatenate([X1, X2], axis = 1)

In [0]:
# splitting data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
# balancing the sample sets as the proportion of labels is unbalanced
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
# defining LSTM model with sigmoid activation functions for prediction
def define_model_LSTM():
    inputs = Input(shape=(length1+length2,))
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    out = Activation('softmax')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [0]:
model = define_model_LSTM()
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 310)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 310, 100)          611900    
_________________________________________________________________
lstm_4 (LSTM)                (None, 310, 16)           7488      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 310, 19)           323       
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 1, 19)             111929    
_________________________________________________________________
reshape_2 (Reshape)          (None, 19)                0         
_________________________________________________________________
activation_2 (Activation)    (None, 19)                0   

In [0]:
# fitting data onto model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/Users/dhwanikapadia/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 20063 samples, validate on 5016 samples
Epoch 1/10
20063/20063 [==============================] - 25s 1ms/step - loss: 0.1990 - accuracy: 0.9474 - val_loss: 0.3313 - val_accuracy: 0.9474
Epoch 2/10
20063/20063 [==============================] - 24s 1ms/step - loss: 0.1976 - accuracy: 0.9474 - val_loss: 0.3589 - val_accuracy: 0.9474
Epoch 3/10
20063/20063 [==============================] - 25s 1ms/step - loss: 0.1948 - accuracy: 0.9474 - val_loss: 0.3735 - val_accuracy: 0.9474
Epoch 4/10
20063/20063 [==============================] - 23s 1ms/step - loss: 0.1882 - accuracy: 0.9479 - val_loss: 0.3692 - val_accuracy: 0.9472
Epoch 5/10
20063/20063 [==============================] - 20s 984us/step - loss: 0.1840 - accuracy: 0.9485 - val_loss: 0.3551 - val_accuracy: 0.9471
Epoch 6/10
20063/20063 [==============================] - 19s 947us/step - loss: 0.1816 - accuracy: 0.9487 - val_loss: 0.3602 - val_accuracy: 0.9472
Epoch 7/10
20063/20063 [==============================] - 17s 871

In [0]:
# evaluating model
model.evaluate(X_test, y_test)

531/531 [==============================] - 0s 683us/step


[0.21767208448798867, 0.9426107406616211]

In [0]:
#predictions for each class from predicting model
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

5     1817
1      678
4      412
2      391
12     257
17     252
6      241
8      240
10     203
0      153
13     143
7      137
11     137
9      135
14      73
3       29
16       4
18       4
dtype: int64

In [0]:
# probability of predicting each class for various records
model.predict(X)

array([[1.09992214e-01, 4.35838699e-02, 6.12151325e-02, ...,
        1.06188265e-04, 3.86776514e-02, 4.35510743e-03],
       [1.64481101e-03, 7.71445036e-02, 4.29336399e-01, ...,
        4.60881332e-04, 1.18156224e-01, 8.77555180e-03],
       [7.52144083e-02, 3.22915837e-02, 5.50202467e-02, ...,
        2.24964431e-04, 2.13452484e-02, 1.99981523e-03],
       ...,
       [6.59008101e-02, 1.10627957e-01, 7.70954266e-02, ...,
        1.86874787e-03, 1.08826131e-01, 2.35029636e-03],
       [7.52144009e-02, 3.22915800e-02, 5.50202429e-02, ...,
        2.24964417e-04, 2.13452447e-02, 1.99981499e-03],
       [1.34782949e-02, 1.41287267e-01, 2.91009005e-02, ...,
        3.02958046e-03, 7.33574778e-02, 5.17956913e-04]], dtype=float32)

In [0]:
# split of classes
pd.Series(y.argmax(axis = -1)).value_counts()

1     1449
17     908
2      511
12     288
6      268
7      260
0      249
8      244
4      241
10     223
14     176
18      96
11      77
9       73
5       73
13      61
3       56
15      44
16       9
dtype: int64

In [0]:
### LSTM Glove

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
# tokenizing the data and integer encoding it

tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
# running the tokenizer function on features

X = tokenizer.texts_to_sequences(X)

In [0]:
# getting vocab size and length for input

length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
# padding the data to make it same size (max size)

X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
# splitting training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
# balancing the dataset to account for disproportionate of labels

resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
# using glove vector to create function that makes word emebeddings

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
# creating an LSTM Glove model

def define_model_LSTM_Glove():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [0]:
model = define_model_LSTM_Glove()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 155, 100)          612000    
_________________________________________________________________
masking_2 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 155, 64)           42240     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_4 (Dense)              (None, 19)                323       
Total params: 688,627
Trainable params: 76,627
Non-trainable params: 612,000
___________________________________________

In [0]:
# fitting model on training data
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 19727 samples, validate on 4932 samples
Epoch 1/10
19727/19727 [==============================] - 45s 2ms/step - loss: 0.2047 - accuracy: 0.9474 - val_loss: 0.2307 - val_accuracy: 0.9474
Epoch 2/10
19727/19727 [==============================] - 43s 2ms/step - loss: 0.2017 - accuracy: 0.9474 - val_loss: 0.2617 - val_accuracy: 0.9474
Epoch 3/10
19727/19727 [==============================] - 43s 2ms/step - loss: 0.1979 - accuracy: 0.9474 - val_loss: 0.3020 - val_accuracy: 0.9474
Epoch 4/10
19727/19727 [==============================] - 43s 2ms/step - loss: 0.1950 - accuracy: 0.9474 - val_loss: 0.3463 - val_accuracy: 0.9474
Epoch 5/10
19727/19727 [==============================] - 43s 2ms/step - loss: 0.1932 - accuracy: 0.9473 - val_loss: 0.3596 - val_accuracy: 0.9474
Epoch 6/10
19727/19727 [==============================] - 43s 2ms/step - loss: 0.1918 - accuracy: 0.9474 - val_loss: 0.3840 - val_accuracy: 0.9474
Epoch 7/10
19727/19727 [==============================] - 43s 2ms/ste

In [0]:
# evaluating model
model.evaluate(X_test, y_test)

531/531 [==============================] - 1s 1ms/step


[0.18758753941108502, 0.9479632377624512]

In [0]:
# making predictions
results = model.predict(X)

In [0]:
# defining a threshold for prediction
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [0]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

1194

In [0]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,beige,black,blue,burgundy,darkbrown,gold,gray,greens,lightbrowns,multi,navy,orange,pinks,purple,red,silver,teal,white,yellow,attribute_value
0,,black,blue,,,,,,,,,,,,,,,white,,"black, blue, white"
1,,black,,,,,,,,,,,,,,,,white,,"black, white"
2,,,,,,,,,,,,,,,,,,,,
3,,black,,,,,,,,,,,,,,,,,,black
4,,black,,,,,,,,,,,,,,,,,,black


In [0]:
results_df['attribute_value'].value_counts()

black, white                   2064
                               1194
black                           993
black, blue, white              395
multi, orange                   342
multi                            80
black, multi                     52
black, blue                      45
blue, white                      27
blue                             26
gold                             19
black, multi, white              17
gold, gray, lightbrowns           7
gray                              7
lightbrowns                       6
orange                            5
pinks                             4
burgundy, gold                    4
navy                              3
burgundy                          3
multi, navy                       2
black, darkbrown                  2
burgundy, gold, lightbrowns       2
blue, navy                        2
blue, multi, orange               1
gold, gray                        1
black, blue, gray                 1
black, navy                 

In [0]:
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
import pickle
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
def get_primarycolor(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [0]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [0]:
get_primarycolor(brand, product_full_name, description, details, brand_category)

,brand,product_full_name,description,details,brand_category,attribute_value
0,frame,les second medium noir,"'minimal , modern styling meet refined luxury ...",NaN,accessory,multi


In [0]:
results=pd.DataFrame(results)
results.to_excel('results_pc.xlsx')

In [0]:
results_df.to_excel('results_df_pc.xlsx')